In [1]:
import pandas as pd
import numpy as np

import mysql.connector

from utils_text import strip_accents

In [2]:
def clean_text(string: str) -> str:
    string = string.lower()
    string = strip_accents(string)
    string = string.strip()
    
    return string    

In [3]:
df_2019 = pd.read_csv('datatran2019.csv', encoding='latin1', sep=';')

In [4]:
df_2020 = pd.read_csv('datatran2020.csv', encoding='latin1', sep=';')

In [5]:
df = pd.concat([df_2019, df_2020])

In [6]:
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,182210.0,2019-01-01,terça-feira,01:30:00,SP,116.0,218,GUARULHOS,Falta de Atenção à Condução,Colisão com objeto estático,...,0,0,0,4,1,"-23,46052014","-46,48772478",SR-SP,DEL6/1,UOP01/SP
1,182211.0,2019-01-01,terça-feira,01:30:00,PR,373.0,"177,3",PONTA GROSSA,Falta de Atenção à Condução,Colisão traseira,...,0,0,0,1,1,"-25,05533957","-50,22776753",SR-PR,DEL7/3,UOP01/PR
2,182212.0,2019-01-01,terça-feira,00:45:00,SC,101.0,16,GARUVA,Animais na Pista,Colisão com objeto estático,...,0,0,0,1,1,"-26,1216","-48,8826",SR-SC,DEL8/3,UOP01/SC
3,182214.0,2019-01-01,terça-feira,00:00:00,CE,20.0,"429,5",FORTALEZA,Ingestão de Substâncias Psicoativas,Colisão com objeto estático,...,1,0,0,1,1,"-3,76999","-38,670063",SR-CE,DEL16/1,UOP02/CE
4,182215.0,2019-01-01,terça-feira,01:00:00,MG,381.0,327,NOVA ERA,Falta de Atenção à Condução,Colisão transversal,...,1,1,0,1,2,"-19,7609","-43,0306",SR-MG,DEL4/3,UOP01/MG


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83154 entries, 0 to 15707
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      83154 non-null  float64
 1   data_inversa            83154 non-null  object 
 2   dia_semana              83154 non-null  object 
 3   horario                 83154 non-null  object 
 4   uf                      83154 non-null  object 
 5   br                      83033 non-null  float64
 6   km                      83033 non-null  object 
 7   municipio               83154 non-null  object 
 8   causa_acidente          83154 non-null  object 
 9   tipo_acidente           83154 non-null  object 
 10  classificacao_acidente  83154 non-null  object 
 11  fase_dia                83154 non-null  object 
 12  sentido_via             83154 non-null  object 
 13  condicao_metereologica  83154 non-null  object 
 14  tipo_pista              83154 non-null

In [8]:
colunas = ["id",
           "data_inversa",
           "horario",
           "uf",                                
           "br",              
           "km",                                
           "municipio",                         
           "causa_acidente",                    
           "tipo_acidente",                     
           "classificacao_acidente",            
           "fase_dia",                          
           "sentido_via",                       
           "condicao_metereologica",            
           "tipo_pista",                        
           "tracado_via",                       
           "pessoas",                            
           "mortos",                             
           "feridos_leves",                      
           "feridos_graves",                     
           "ilesos",                             
           "ignorados",                          
           "feridos",                            
           "veiculos"]

In [9]:
df = df[colunas]

In [10]:
df.head()

,id,data_inversa,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,tipo_pista,tracado_via,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,182210.0,2019-01-01,01:30:00,SP,116.0,218,GUARULHOS,Falta de Atenção à Condução,Colisão com objeto estático,Com Vítimas Feridas,...,Múltipla,Curva,4,0,4,0,0,0,4,1
1,182211.0,2019-01-01,01:30:00,PR,373.0,"177,3",PONTA GROSSA,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,...,Dupla,Reta,1,0,1,0,0,0,1,1
2,182212.0,2019-01-01,00:45:00,SC,101.0,16,GARUVA,Animais na Pista,Colisão com objeto estático,Com Vítimas Feridas,...,Dupla,Reta,1,0,1,0,0,0,1,1
3,182214.0,2019-01-01,00:00:00,CE,20.0,"429,5",FORTALEZA,Ingestão de Substâncias Psicoativas,Colisão com objeto estático,Com Vítimas Feridas,...,Dupla,Viaduto,1,0,0,1,0,0,1,1
4,182215.0,2019-01-01,01:00:00,MG,381.0,327,NOVA ERA,Falta de Atenção à Condução,Colisão transversal,Com Vítimas Feridas,...,Simples,Interseção de vias,2,0,0,1,1,0,1,2


# Tratamento

In [11]:
# Tratamento/separação de tempo(dia)
df['data_hora'] = df['data_inversa'].map(str) + ' ' + df['horario']
df['data_hora'] = pd.to_datetime(df['data_hora'])
df['ano'] = df['data_hora'].dt.year
df['mes'] = df['data_hora'].dt.month
df['hora'] = df['data_hora'].dt.hour
df.drop(['data_inversa', 'horario'], axis=1, inplace = True)

In [12]:
df.head(3)

,id,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,...,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,data_hora,ano,mes,hora
0,182210.0,SP,116.0,218,GUARULHOS,Falta de Atenção à Condução,Colisão com objeto estático,Com Vítimas Feridas,Plena Noite,Crescente,...,4,0,0,0,4,1,2019-01-01 01:30:00,2019,1,1
1,182211.0,PR,373.0,"177,3",PONTA GROSSA,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Plena Noite,Crescente,...,1,0,0,0,1,1,2019-01-01 01:30:00,2019,1,1
2,182212.0,SC,101.0,16,GARUVA,Animais na Pista,Colisão com objeto estático,Com Vítimas Feridas,Plena Noite,Decrescente,...,1,0,0,0,1,1,2019-01-01 00:45:00,2019,1,0


In [13]:
# Drop de faltantes
df = df.dropna()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83033 entries, 0 to 15707
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      83033 non-null  float64       
 1   uf                      83033 non-null  object        
 2   br                      83033 non-null  float64       
 3   km                      83033 non-null  object        
 4   municipio               83033 non-null  object        
 5   causa_acidente          83033 non-null  object        
 6   tipo_acidente           83033 non-null  object        
 7   classificacao_acidente  83033 non-null  object        
 8   fase_dia                83033 non-null  object        
 9   sentido_via             83033 non-null  object        
 10  condicao_metereologica  83033 non-null  object        
 11  tipo_pista              83033 non-null  object        
 12  tracado_via             83033 non-null  object

In [15]:
# Tratamento de texto: acentos e simbolos, lower case e strip
for column in df.columns:
    if df[column].dtypes == 'object':
        df[column] = df[column].apply(clean_text)

### RESULTADO

In [16]:
df.head()

,id,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,...,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,data_hora,ano,mes,hora
0,182210.0,sp,116.0,218,guarulhos,falta de atencao a conducao,colisao com objeto estatico,com vitimas feridas,plena noite,crescente,...,4,0,0,0,4,1,2019-01-01 01:30:00,2019,1,1
1,182211.0,pr,373.0,"177,3",ponta grossa,falta de atencao a conducao,colisao traseira,com vitimas feridas,plena noite,crescente,...,1,0,0,0,1,1,2019-01-01 01:30:00,2019,1,1
2,182212.0,sc,101.0,16,garuva,animais na pista,colisao com objeto estatico,com vitimas feridas,plena noite,decrescente,...,1,0,0,0,1,1,2019-01-01 00:45:00,2019,1,0
3,182214.0,ce,20.0,"429,5",fortaleza,ingestao de substancias psicoativas,colisao com objeto estatico,com vitimas feridas,plena noite,decrescente,...,0,1,0,0,1,1,2019-01-01 00:00:00,2019,1,0
4,182215.0,mg,381.0,327,nova era,falta de atencao a conducao,colisao transversal,com vitimas feridas,plena noite,decrescente,...,0,1,1,0,1,2,2019-01-01 01:00:00,2019,1,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83033 entries, 0 to 15707
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      83033 non-null  float64       
 1   uf                      83033 non-null  object        
 2   br                      83033 non-null  float64       
 3   km                      83033 non-null  object        
 4   municipio               83033 non-null  object        
 5   causa_acidente          83033 non-null  object        
 6   tipo_acidente           83033 non-null  object        
 7   classificacao_acidente  83033 non-null  object        
 8   fase_dia                83033 non-null  object        
 9   sentido_via             83033 non-null  object        
 10  condicao_metereologica  83033 non-null  object        
 11  tipo_pista              83033 non-null  object        
 12  tracado_via             83033 non-null  object

# Criação de DFs para ETL

In [18]:
# uf
df_uf = list(df['uf'].unique())
df_uf

['sp',
 'pr',
 'sc',
 'ce',
 'mg',
 'ms',
 'go',
 'rs',
 'rj',
 'pb',
 'rn',
 'pe',
 'se',
 'df',
 'to',
 'mt',
 'es',
 'ro',
 'pa',
 'al',
 'am',
 'ma',
 'ba',
 'pi',
 'ap',
 'rr',
 'ac']

In [19]:
# municipio
df_municipio = list(df['municipio'].unique())
df_municipio

['guarulhos',
 'ponta grossa',
 'garuva',
 'fortaleza',
 'nova era',
 'gaspar',
 'coxim',
 'catalao',
 'goiania',
 'boa vista das missoes',
 'uberlandia',
 'ubatuba',
 'maringa',
 'betim',
 'duque de caxias',
 'cunha pora',
 'cajazeiras',
 'macaiba',
 'natal',
 'ipuiuna',
 'angra dos reis',
 'rio verde de mato grosso',
 'douradina',
 'sete lagoas',
 'goiana',
 'estancia',
 'brasilia',
 'joao pinheiro',
 'papanduva',
 'itaborai',
 'indaial',
 'tibagi',
 'carlos barbosa',
 'russas',
 'gurupi',
 'icara',
 'igaratinga',
 'rio verde',
 'rosario do sul',
 'campo bonito',
 'agua boa',
 'cascavel',
 'sooretama',
 'catingueira',
 'ouro preto do oeste',
 'dom eliseu',
 'palhoca',
 'diamantino',
 'rio largo',
 'itatiaiucu',
 'porto velho',
 'lages',
 'manaus',
 'caxias',
 'rio real',
 'formosa',
 'maracanau',
 'mangaratiba',
 'tres forquilhas',
 'domingos martins',
 'sao paulo',
 'cacimbinhas',
 'oliveira fortes',
 'amelia rodrigues',
 'altos',
 'feira de santana',
 'araquari',
 'mucuri',
 'canoi

In [20]:
# causa_acidente
df_causa_acidente = list(df['causa_acidente'].unique())
df_causa_acidente

['falta de atencao a conducao',
 'animais na pista',
 'ingestao de substancias psicoativas',
 'desobediencia as normas de transito pelo condutor',
 'desobediencia as normas de transito pelo pedestre',
 'ingestao de alcool',
 'velocidade incompativel',
 'ingestao de alcool e/ou substancias psicoativas pelo pedestre',
 'pista escorregadia',
 'condutor dormindo',
 'defeito mecanico no veiculo',
 'sinalizacao da via insuficiente ou inadequada',
 'defeito na via',
 'nao guardar distancia de seguranca',
 'objeto estatico sobre o leito carrocavel',
 'restricao de visibilidade',
 'falta de atencao do pedestre',
 'ultrapassagem indevida',
 'avarias e/ou desgaste excessivo no pneu',
 'mal subito',
 'agressao externa',
 'carga excessiva e/ou mal acondicionada',
 'fenomenos da natureza',
 'deficiencia ou nao acionamento do sistema de iluminacao/sinalizacao do veiculo']

In [21]:
# tipo_acidente
df_tipo_acidente = list(df['tipo_acidente'].unique())
df_tipo_acidente

['colisao com objeto estatico',
 'colisao traseira',
 'colisao transversal',
 'saida de leito carrocavel',
 'atropelamento de animal',
 'atropelamento de pedestre',
 'colisao frontal',
 'colisao lateral',
 'queda de ocupante de veiculo',
 'incendio',
 'capotamento',
 'engavetamento',
 'tombamento',
 'derramamento de carga',
 'colisao com objeto em movimento',
 'danos eventuais']

In [22]:
# classificacao_acidente
df_classificacao_acidente = list(df['classificacao_acidente'].unique())
df_classificacao_acidente

['com vitimas feridas', 'sem vitimas', 'com vitimas fatais']

In [23]:
# fase_dia
df_fase_dia = list(df['fase_dia'].unique())
df_fase_dia

['plena noite', 'amanhecer', 'pleno dia', 'anoitecer']

In [24]:
# sentido_via
df_sentido_via = list(df['sentido_via'].unique())
df_sentido_via

['crescente', 'decrescente']

In [25]:
# condicao_metereologica
df_condicao_metereologica = list(df['condicao_metereologica'].unique())
df_condicao_metereologica

['ceu claro',
 'nublado',
 'chuva',
 'garoa/chuvisco',
 'ignorado',
 'nevoeiro/neblina',
 'vento',
 'sol',
 'granizo',
 'neve']

In [26]:
# tipo_pista
df_tipo_pista = list(df['tipo_pista'].unique())
df_tipo_pista

['multipla', 'dupla', 'simples']

In [27]:
# tracado_via
df_tracado_via = list(df['tracado_via'].unique())
df_tracado_via

['curva',
 'reta',
 'viaduto',
 'intersecao de vias',
 'nao informado',
 'rotatoria',
 'desvio temporario',
 'retorno regulamentado',
 'tunel',
 'ponte']

### DF FINAL

In [28]:
df_final = df.copy()
df_final = df_final.drop('ano', axis=1)
df_final = df_final.drop('mes', axis=1)
df_final = df_final.drop('hora', axis=1)
df_final = df_final.reset_index(drop=True)

In [29]:
# Troca de , para . nos numeros nao inteiros
df_final['km'] = df_final['km'].apply(lambda x: float(x.replace(',','.')))

In [30]:
dict_df = {'uf': df_uf,
           'municipio': df_municipio,
           'causa_acidente': df_causa_acidente,
           'tipo_acidente': df_tipo_acidente,
           'classificacao_acidente': df_classificacao_acidente,
           'fase_dia': df_fase_dia,
           'sentido_via': df_sentido_via,
           'condicao_metereologica': df_condicao_metereologica,
           'tipo_pista': df_tipo_pista,
           'tracado_via': df_tracado_via}

In [31]:
for column in list(dict_df.keys()):
    for i, value in enumerate(df[column]):
        index = dict_df[column].index(value)
        index +=1
        df_final[column][i] = index

<ipython-input-31-287be7b1512e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[column][i] = index


In [32]:
df_final.head()

,id,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,...,tracado_via,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,data_hora
0,182210.0,1,116.0,218.0,1,1,1,1,1,1,...,1,4,0,4,0,0,0,4,1,2019-01-01 01:30:00
1,182211.0,2,373.0,177.3,2,1,2,1,1,1,...,2,1,0,1,0,0,0,1,1,2019-01-01 01:30:00
2,182212.0,3,101.0,16.0,3,2,1,1,1,2,...,2,1,0,1,0,0,0,1,1,2019-01-01 00:45:00
3,182214.0,4,20.0,429.5,4,3,1,1,1,2,...,3,1,0,0,1,0,0,1,1,2019-01-01 00:00:00
4,182215.0,5,381.0,327.0,5,1,3,1,1,2,...,4,2,0,0,1,1,0,1,2,2019-01-01 01:00:00


In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83033 entries, 0 to 83032
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      83033 non-null  float64       
 1   uf                      83033 non-null  object        
 2   br                      83033 non-null  float64       
 3   km                      83033 non-null  float64       
 4   municipio               83033 non-null  object        
 5   causa_acidente          83033 non-null  object        
 6   tipo_acidente           83033 non-null  object        
 7   classificacao_acidente  83033 non-null  object        
 8   fase_dia                83033 non-null  object        
 9   sentido_via             83033 non-null  object        
 10  condicao_metereologica  83033 non-null  object        
 11  tipo_pista              83033 non-null  object        
 12  tracado_via             83033 non-null  object

# ETL

In [34]:
cnx = mysql.connector.connect(user='root', password='root',
                              host='localhost',
                              database='acidentes_db')

In [35]:
cursor = cnx.cursor()

In [36]:
dict_colunas = {'CAUSA_ACIDENTE': df_causa_acidente,
                'CLF_ACIDENTE': df_classificacao_acidente, 
                'COND_METEREOLOGICA': df_condicao_metereologica, 
                'FASE_DIA': df_fase_dia, 
                'MUNICIPIO': df_municipio, 
                'SENTIDO_VIA': df_sentido_via, 
                'TIPO_ACIDENTE': df_tipo_acidente, 
                'TIPO_PISTA': df_tipo_pista, 
                'UF': df_uf, 
                'TRACADO_VIA': df_tracado_via}

In [37]:
# Colunas normais
for name, df in dict_colunas.items():
    for value in df:
        query = f"INSERT INTO {name} "\
                 f"(NOME)" \
                 f"VALUES ({value!r})"
        cursor.execute(query)

In [38]:
cnx.commit()

In [39]:
# Coluna acidente
for index, name in df_final.iterrows():
    query = f"INSERT INTO ACIDENTE"\
                 f"(ID_ACIDENTE, ID_UF, BR, KM, ID_MUNICIPIO, ID_CAUSA_ACIDENTE, ID_TIPO_ACIDENTE, ID_CLF_ACIDENTE, ID_FASE_DIA, ID_SENTIDO_VIA, ID_COND_METEREOLOGICA, ID_TIPO_PISTA, ID_TRACADO_VIA, NUM_PESSOAS, NUM_MORTOS, NUM_FERIDOS_LEVES, NUM_FERIDOS_GRAVES, NUM_ILESOS, NUM_IGNORADOS, NUM_FERIDOS, NUM_VEICULOS, DATA)" \
                 f"VALUES ({name['id']!r}, {name['uf']!r}, {name['br']!r},{name['km']!r}, {name['municipio']!r}, {name['causa_acidente']!r}, {name['tipo_acidente']!r}, {name['classificacao_acidente']!r}, {name['fase_dia']!r}, {name['sentido_via']!r}, {name['condicao_metereologica']!r}, {name['tipo_pista']!r}, {name['tracado_via']!r}, {name['pessoas']!r}, {name['mortos']!r}, {name['feridos_leves']!r}, {name['feridos_graves']!r}, {name['ilesos']!r}, {name['ignorados']!r}, {name['feridos']!r}, {name['veiculos']!r}, {name['data_hora']!r})"
    cursor.execute(query)

In [40]:
cnx.commit()

In [41]:
cnx.close()